In [1]:
# 방대한 양의 데이터는 한 번에 전부를 학습하면서 경사 하강법을 수행하면 수행 속도가 매우 느리게 진행될 것.
# 많은 계산량이 필요하므로 메모리도 많이 잡아먹는다.
# --> 전체 데이터를 더 작은 단위로 나눠서 그 단위로 학습하는 미니 배치 경사 하강법 개념이 나오게 되었다.
# (업데이트 한 번에 전체 데이터를 사용하는 것을 배치 경사 하강법, 미니 배치를 사용하면 미니 배치 경사 하강법)

# Epoch, Mini batch, Iteration에 대해
# Epoch : 전체 데이터를 몇 번 사용할 것인가
# Mini batch : 전체 데이터를 몇 개의 미니 배치로 쪼갤 것인가
# Iteration : 그래서 몇 번 업데이트 하는가(Epoch * Minibatch)
# ** 배치 크기(미니 배치 하나의 사이즈)는 일반적으로 2의 제곱수. why? CPU, GPU의 메모리가 2의 배수이기 때문에
# 2의 배수로 할 경우 데이터 송수신 효율을 높일 수 있다.

In [3]:
# 데이터를 쉽게 다루는 도구로서, 데이터셋(Dataset)과 데이터로더(DataLoader)를 제공한다.
# 미니 배치 학습, 데이터 셔플, 병렬 처리를 지원하는 기능
# 기본적 사용 방법 :
## Dataset 정의 --> DataLoader에 전달.
# 여기서는 텐서를 입력받아서 Dataset의 형태로 변환해주는 TensorDataset을 사용한다.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# TensorDataset, DataLoader 임포트
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [5]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [6]:
# x_train, y_train Tensor를 TensorDataset에 인자로 전달하여, dataset으로 저장
dataset = TensorDataset(x_train, y_train)

In [7]:
# Dataset을 만들었다면, DataLoader를 사용 가능하다. 데이터로더는 2개의 인자를 입력받는다.
# 1. 데이터셋
# 2. 미니 배치 size(통상적으로 2의 제곱수)
# 3. (optional): shuffle = True를 선택하면 Epoch마다 데이터셋을 섞어서, 데이터 학습 순서를 바꾼다. 좋아보임

dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [9]:
model = nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [11]:
nb_epochs = 20
for epoch in range(nb_epochs + 1): # 각 에포크마다.
    for batch_idx, samples in enumerate(dataloader): # 데이터로더에서 지원하는 각 미니배치 개수만큼 for문
        print(batch_idx)
        print(samples)
        x_train, y_train = samples

        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print(f"{epoch}, {batch_idx}, {cost}")

0
[tensor([[ 96.,  98., 100.],
        [ 73.,  80.,  75.]]), tensor([[196.],
        [152.]])]
0, 0, 8970.6875
1
[tensor([[93., 88., 93.],
        [89., 91., 90.]]), tensor([[185.],
        [180.]])]
0, 1, 2943.9248046875
2
[tensor([[73., 66., 70.]]), tensor([[142.]])]
0, 2, 530.4251098632812
0
[tensor([[73., 80., 75.],
        [73., 66., 70.]]), tensor([[152.],
        [142.]])]
1, 0, 224.3378143310547
1
[tensor([[ 96.,  98., 100.],
        [ 89.,  91.,  90.]]), tensor([[196.],
        [180.]])]
1, 1, 145.4307403564453
2
[tensor([[93., 88., 93.]]), tensor([[185.]])]
1, 2, 71.75151824951172
0
[tensor([[73., 80., 75.],
        [93., 88., 93.]]), tensor([[152.],
        [185.]])]
2, 0, 8.944786071777344
1
[tensor([[73., 66., 70.],
        [89., 91., 90.]]), tensor([[142.],
        [180.]])]
2, 1, 4.62497091293335
2
[tensor([[ 96.,  98., 100.]]), tensor([[196.]])]
2, 2, 0.025081364437937737
0
[tensor([[ 93.,  88.,  93.],
        [ 96.,  98., 100.]]), tensor([[185.],
        [196.]])]
3, 0

In [12]:
# 5개 있기 때문에, 마지막 mini batch는 한 개의 데이터를 가짐.

In [13]:
# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[153.4668]], grad_fn=<AddmmBackward0>)
